<a href="https://colab.research.google.com/github/daniloas1992/xbi_classification_colab/blob/main/xbi_classification_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np

##**Configuração para executar com GPU em Ambiente de Execução Local**

Desabilitar a exibição de logs da GPU (Exibe apenas erros):

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Limitar o uso de memória da GPU, pois se atingir o limite causa erro na execução do treinamento:

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:

    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Exibir a versão do Tensorflow

In [4]:
print(tf.__version__)

2.1.0


##**Obter informações do hardware**

Exibir informações sobre GPU:

In [ ]:
device_name = tf.test.gpu_device_name()
device_lib.list_local_devices()

Exibir informações sobre CPU e Memória:

In [ ]:
!cat /proc/cpuinfo
!cat /proc/meminfo

Verificar se a GPU está disponível:

In [7]:
if device_name != '/device:GPU:0':
    print('ATENTION: GPU device not found. CPU will be used!')
else:  
    print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


##**Configuração Inicial do classificador**

In [8]:
extractor_name = 'image_diff_extractor'
class_attr = 'internal'
sampler_name = 'none'
n_splits = 24

#**Configurar dataset**

Criar lista com nome das colunas do dataset que não serão utilizadas pelo classificador CNN:

In [9]:
exclude_columns = ['id','tagName','childsNumber','textLength','baseBrowser','targetBrowser','baseDPI','targetDPI','baseScreenshot','targetScreenshot','baseX',
'targetX','baseY','targetY','baseHeight','targetHeight','baseWidth','targetWidth','baseParentX','targetParentX','baseParentY','targetParentY','imageDiff',
'chiSquared','baseDeviceWidth','targetDeviceWidth','baseViewportWidth','targetViewportWidth','xpath','baseXpath','targetXpath','phash','basePreviousSiblingLeft',
'targetPreviousSiblingLeft','basePreviousSiblingTop','targetPreviousSiblingTop','baseNextSiblingLeft','targetNextSiblingLeft','baseNextSiblingTop',
'targetNextSiblingTop','baseTextNodes','targetTextNodes','baseFontFamily','targetFontFamily','base_bin1','base_bin2','base_bin3','base_bin4','base_bin5',
'base_bin6','base_bin7','base_bin8','base_bin9','base_bin10','target_bin1','target_bin2','target_bin3','target_bin4','target_bin5','target_bin6','target_bin7',
'target_bin8','target_bin9','target_bin10','emd','ssim','mse','psnr']

Importar o dataset sem as colunas desnecessárias:

In [ ]:
#dataset_path = '/content/32x32-dataset.csv' # Carregar o dataset se estiver utilizando o ambiente de execução do Colab. Precisa fazer upload do arquivo.
dataset_path = '/home/danilo/Mestrado/xbi_configurar_dataset/ConvertArffToCsv/32x32_Corrigido.csv' # Carregar o dataset do ambiente de execução local
data = pd.read_csv(dataset_path, usecols= lambda column : column not in exclude_columns)

Cria variáveis com os parâmetros das imagens do dataset:

In [ ]:
image_width = 32
image_height = 32
image_color_channel = 3 #RGB
image_color_channel_size = 255
image_size = (image_width, image_height)
image_shape = image_size + (image_color_channel, )